In [2]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [3]:
url1 = "http://lfcstats.co.uk/20162017results.html"
url2 = "http://lfcstats.co.uk/20162017premierleagueresults.html"
header = {
        'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_2) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/47.0.2526.80 Safari/537.36'
    }

In [5]:
data = requests.get(url1, headers = header).content
soup = BeautifulSoup(data)

/Users/ZHHR/anaconda2/lib/python2.7/site-packages/bs4/__init__.py:181: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 174 of the file /Users/ZHHR/anaconda2/lib/python2.7/runpy.py. To get rid of this warning, change code that looks like this:

 BeautifulSoup([your markup])

to this:

 BeautifulSoup([your markup], "lxml")

  markup_type=markup_type))


In [6]:
def lfc_crawler_16_17(url):
    header = {
        'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_2) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/47.0.2526.80 Safari/537.36'
    }
    data = requests.get(url, headers = header).content
    soup = BeautifulSoup(data)
    data_raw = soup.find("section", attrs={"id": "content"})
    items = data_raw.find_all("table")
    df = pd.DataFrame(index=np.arange(len(items)), columns=['date','home', 'away', 'score', 'points'])
    count = 0
    for item in items:
        tmp1 = item.find("span", attrs={"class": "style4 style4"} )
        tmp2 = item.find_all('strong')
        tmp3 = item.find('a')
        if tmp1 and tmp2 and tmp3:
            df.date[count] = tmp1.getText()
            df.home[count] = tmp2[0].getText()
            df.away[count] = tmp2[1].getText()
            df.score[count] = tmp3.getText()
            count += 1
    df = df[df.home.notnull()]
    for i in xrange(len(df)):
        tmp = df.score[i]
        if df.home[i] == 'Liverpool':
            if tmp[0] > tmp[2]:
                df.points[i] = 3
            elif tmp[0] == tmp[2]:
                df.points[i] = 1
            else:
                df.points[i] = 0
        else:
            if tmp[0] < tmp[2]:
                df.points[i] = 3
            elif tmp[0] == tmp[2]:
                df.points[i] = 1
            else:
                df.points[i] = 0       
    return df

In [7]:
df_premier_league = lfc_crawler_16_17(url2)
df_all_games = lfc_crawler_16_17(url1)

In [6]:
df_premier_league.to_csv('lfc_pl_16_17.csv')
df_all_games.to_csv('lfc_all_16_17.csv')

In [ ]:
def DataFilter_liv(df1):
    df = df1.copy(deep=True)
    df['Vic'] = np.NAN
    df['GF'] = np.NAN
    df['GA'] = np.NAN
    df['GD'] = np.NAN
    df['ExpGf'] = np.NAN
    df['ExpGa'] = np.NAN
    df['ExpGd'] = np.NAN
    df['ExpPPM'] = np.NAN
    df['ExpRate'] = np.NAN
    for i in xrange(len(df)):
        if i == 888:
            df.score[i] = "2:2"
        tmp = df.score[i]
        #chapter_2.GF[i] = int(tmp[0])
        df.loc[i, 'GF'] = int(tmp[0])
        df.loc[i, 'GA'] = int(tmp[2])
        df.loc[i, 'GD'] = int(tmp[0]) - int(tmp[2])
        df.loc[i, 'Vic'] = float((tmp[0] > tmp[2]))
    df['ExpGf'] = df.GF.cumsum()
    df['ExpGa'] = df.GA.cumsum()
    df['ExpGd'] = df.GD.cumsum()
    df['ExpPPM'] = df.points.cumsum()
    df['ExpRate'] = df.Vic.cumsum()
    for i in xrange(len(df)):
        df.loc[i, 'ExpGf'] = df['ExpGf'][i] / (i+1)
        df.loc[i, 'ExpGa'] = df['ExpGa'][i] / (i+1)
        df.loc[i, 'ExpGd'] = df['ExpGd'][i] / (i+1)
        df.loc[i, 'ExpPPM'] = df['ExpPPM'][i] / (i+1)
        df.loc[i, 'ExpRate'] = df['ExpRate'][i] / (i+1)

    return df
df00 = pd.read_csv('./data/liverpool-fc/liverpool-fc_1998_1999.csv', index_col=0, dtype={'points': float})
df0 = pd.read_csv('./data/liverpool-fc/liverpool-fc_1999_2000.csv', index_col=0, dtype={'points': float})

df1 = pd.read_csv('./data/liverpool-fc/liverpool-fc_2000_2001.csv', index_col=0, dtype={'points': float})
df2 = pd.read_csv('./data/liverpool-fc/liverpool-fc_2001_2002.csv', index_col=0, dtype={'points': float})
df3 = pd.read_csv('./data/liverpool-fc/liverpool-fc_2002_2003.csv', index_col=0, dtype={'points': float})
df4 = pd.read_csv('./data/liverpool-fc/liverpool-fc_2003_2004.csv', index_col=0, dtype={'points': float})

df5 = pd.read_csv('./data/liverpool-fc/liverpool-fc_2004_2005.csv', index_col=0, dtype={'points': float})
df6 = pd.read_csv('./data/liverpool-fc/liverpool-fc_2005_2006.csv', index_col=0, dtype={'points': float})
df7 = pd.read_csv('./data/liverpool-fc/liverpool-fc_2006_2007.csv', index_col=0, dtype={'points': float})
df8 = pd.read_csv('./data/liverpool-fc/liverpool-fc_2007_2008.csv', index_col=0, dtype={'points': float})

df9 = pd.read_csv('./data/liverpool-fc/liverpool-fc_2008_2009.csv', index_col=0, dtype={'points': float})
df10 = pd.read_csv('./data/liverpool-fc/liverpool-fc_2009_2010.csv', index_col=0, dtype={'points': float})
df11 = pd.read_csv('./data/liverpool-fc/liverpool-fc_2010_2011.csv', index_col=0, dtype={'points': float})
df12 = pd.read_csv('./data/liverpool-fc/liverpool-fc_2011_2012.csv', index_col=0, dtype={'points': float})
df13 = pd.read_csv('./data/liverpool-fc/liverpool-fc_2012_2013.csv', index_col=0, dtype={'points': float})
df14 = pd.read_csv('./data/liverpool-fc/liverpool-fc_2013_2014.csv', index_col=0, dtype={'points': float})
df15 = pd.read_csv('./data/liverpool-fc/liverpool-fc_2014_2015.csv', index_col=0, dtype={'points': float})
df16 = pd.read_csv('./data/liverpool-fc/liverpool-fc_2015_2016.csv', index_col=0, dtype={'points': float})
df17 = pd.read_csv('./data/liverpool-fc/liverpool-fc_2016_2017.csv', index_col=0, dtype={'points': float})
df18 = pd.read_csv('./data/liverpool-fc/liverpool-fc_2017_2018.csv', index_col=0, dtype={'points': float})

liverpool_raw = pd.concat([df00, df0, df1, df2, df3, df4, df5, df6, df7, df8, df9, df10, df11, 
                           df12, df13, df14, df15, df16, df17, df18], ignore_index=True)
#liverpool_raw = pd.concat([df1, df2, df3, df4, df5,
#                          df6, df7, df8, df9, df10, df11], ignore_index=True)

liverpool_raw.date = pd.to_datetime(liverpool_raw.date)
liverpool_raw = liverpool_raw.sort_values(by='date')
#mask = (chapter_4_raw.date >= start_4) & (chapter_4_raw.date <= end_4)

#chapter_4_raw = chapter_4_raw.loc[mask]
liverpool_raw = liverpool_raw.reset_index(drop=True)

In [ ]:
liverpool_raw = DataFilter_liv(liverpool_raw)
liverpool_raw.head(5)
print len(liverpool_raw)

In [ ]:
# chelsea_metric = chapter_4.GA.rolling(window=50).mean()
# arsenal_metric = arsenal_raw.GA.rolling(window=50).mean()
# manu_metric = manU_raw.Vic.rolling(window=50).mean()
liv_metric = liverpool_raw.GD.rolling(window=30).mean()

plt.plot(liverpool_raw.date, liv_metric, 'orange')

# plt.plot(chapter_4.date, chelsea_metric, 'orange')
# plt.plot(arsenal_raw.date, arsenal_metric, color='rosybrown')
# plt.plot(manU_raw.date, manu_metric, 'dodgerblue')
# plt.ylim([.3, 1.6])
# plt.ylabel('GA')
# plt.title('Chelsea FC GA(2000-2011)', dict(size=20))

# plt.axvline(start_4, c='k', ls='--')
# plt.axvline(end_4, c='k', ls='--')

# plt.text(start_4-datetime.timedelta(days=890), .4, "Before Mourinho", dict(size=18), 
#              color='black', family='bold')
# plt.text(start_4, .4, "               Mourinho", dict(size=18), 
#              color='black', family='bold')
# plt.text(end_4, .4, "                    After Mourinho", dict(size=18), 
#              color='black', family='bold')

# plt.legend(('Chelsea FC', 'Arsenal FC', 'Manchester United'), shadow=True, loc=(0.01, 0.85))
plt.show()